# Invoke Bedrock with HealthScribe Summary

> *This notebook is compatible with SageMaker kernels `Data Science 3.0` or `conda_python3` on an `ml.t3.medium` instance*

## Setup
Before running the remaining cells of this notebook, we need to confirm you have a version of boto3 that supports Bedrock. Any version newer than **`1.28.58`** should be fine. Run the following cell to check the installed version. 

In [1]:
!pip list | grep boto3

boto3                         1.34.158


(Optional) Only run this cell if you have an older version of boto3 that does not support Bedrock.

In [2]:
!pip install botocore boto3 --upgrade

#### List available models in Bedrock

This cell calls **`list_foundation_models`** API in Bedrock filtering only models provided by **`Anthropic`**.

In [3]:
import boto3, json

# connect to AWS Bedrock service is the us-west-2 region
bedrock = boto3.client('bedrock', 'us-west-2') 
# get only models from Anthropic
response = bedrock.list_foundation_models(byProvider='anthropic')
print(json.dumps(response.get('modelSummaries'), indent=2))

[
  {
    "modelArn": "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-instant-v1:2:100k",
    "modelId": "anthropic.claude-instant-v1:2:100k",
    "modelName": "Claude Instant",
    "providerName": "Anthropic",
    "inputModalities": [
      "TEXT"
    ],
    "outputModalities": [
      "TEXT"
    ],
    "responseStreamingSupported": true,
    "customizationsSupported": [],
    "inferenceTypesSupported": [
      "PROVISIONED"
    ],
    "modelLifecycle": {
      "status": "ACTIVE"
    }
  },
  {
    "modelArn": "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-instant-v1",
    "modelId": "anthropic.claude-instant-v1",
    "modelName": "Claude Instant",
    "providerName": "Anthropic",
    "inputModalities": [
      "TEXT"
    ],
    "outputModalities": [
      "TEXT"
    ],
    "responseStreamingSupported": true,
    "customizationsSupported": [],
    "inferenceTypesSupported": [
      "ON_DEMAND"
    ],
    "modelLifecycle": {
      "status": "ACTIVE"
    }
 

---

## 3. Combining HealthScribe with Bedrock

In this step we are going to invoke model **`Claude v3 Sonnet`** from **`Anthropic`** on Amazon Bedrock asking the model to provide ICD-10 codes and procedure codes based on the plan generated by AWS HealthScribe. This could be an alternative solution to using Comprehend Medical, but also complements what Comprehend provides by enabling you to extract CPT codes that are currently not supported by the service.

In [4]:
bedrock_runtime = boto3.client('bedrock-runtime', 'us-west-2')

# retrieve plan from previous notebook
%store -r plan
print(plan)


Lower back pain
  - Start physical therapy for a minimum of 4 weeks, ideally 6 weeks, to help relieve the lower back pain without waiting for the X-ray results. 

  - Take an X-ray of the lower back to check for any underlying issues causing the pain.

  - Continue taking over-the-counter pain medications as needed in the meantime.

  - Use a chair with lumbar support or position a pillow for lumbar support while sitting for long periods.

Sleep
  - Improve sleep habits by separating from the toddler at night to allow uninterrupted sleep and a more comfortable sleeping position.

Exercise
  - Start light exercises like walking or yoga to improve posture and handle the pain.

Follow up
  - Reassess the situation after physical therapy - if pain does not improve, order a CT scan of the lower back to further investigate the likely cause of sciatica (pinched nerve in the lower back radiating pain to legs).



This helper function encapsulates Bedrock **`invoke_model`** API call:

In [5]:
def generate_message(prompt, max_tokens, temperature):
    
    # define model id, this can be found from the output of the available models above
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    user_message =  {"role": "user", "content": prompt}
    messages = [user_message]
    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temperature
        }
    )
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
    return response_body

Let's first invoke **`Claude v3 Sonnet`** with a simple test prompt:

In [6]:
prompt = "what is the CPT code for an x-ray of their lower back?"

response_body = generate_message(prompt, 1000, 0)

print(response_body.get('content')[0].get('text'))

The CPT (Current Procedural Terminology) code for an X-ray of the lower back (lumbar spine) is:

72100 - Radiologic examination, spine, lumbosacral; anteroposterior and lateral views

This code covers a standard two-view X-ray of the lumbar spine, which includes an anteroposterior (AP) view and a lateral view.

If additional views or projections are needed, there are separate CPT codes for those, such as:

72110 - Radiologic examination, spine, lumbosacral; complete, including bending views, minimum of 6 views
72114 - Radiologic examination, spine, lumbosacral; complete, including oblique and flexion and extension views

The CPT codes provide a standardized way for healthcare providers to report medical services and procedures to insurance companies for billing purposes. The specific code used depends on the exact type and extent of the X-ray examination performed.


### 3.1. Send HealthScribe Summary Plan to Bedrock to Extract ICD-10 Codes

In this step we are going to invoke model **`Claude v3 Sonnet`** to provide ICD-10 codes for diagnostic components mentioned in the plan generated by HealthScribe like we did in the previous notebook.

In [7]:
prompt = f"""

For each one of the bullet points defined in <plan></plan> XML tags between triple backslashes, 
list the most common ICD-10 codes applicable. If there are no codes for a recommendation, do not include them in your response.
Present your output using the same JSON format used by Amazon Comprehend Medical.

<plan>{plan}</plan>

"""

response_body = generate_message(prompt, 1000, 0)

print(response_body.get('content')[0].get('text'))

```json
{
  "Entities": [
    {
      "Id": 0,
      "Text": "Lower back pain",
      "Category": "MEDICAL_CONDITION",
      "Traits": [
        {
          "Name": "ICD10CMCodes",
          "Score": 1.0,
          "Values": [
            "M54.5"
          ]
        }
      ]
    },
    {
      "Id": 1,
      "Text": "Start physical therapy for a minimum of 4 weeks, ideally 6 weeks, to help relieve the lower back pain without waiting for the X-ray results",
      "Category": "TREATMENT",
      "Traits": []
    },
    {
      "Id": 2,
      "Text": "Take an X-ray of the lower back to check for any underlying issues causing the pain",
      "Category": "TEST",
      "Traits": []
    },
    {
      "Id": 3,
      "Text": "Continue taking over-the-counter pain medications as needed in the meantime",
      "Category": "TREATMENT",
      "Traits": []
    },
    {
      "Id": 4,
      "Text": "Use a chair with lumbar support or position a pillow for lumbar support while sitting for long perio

### 3.2. Send HealthScribe Summary Plan to Bedrock to Extract CPT Codes

Now we are going to invoke model **`Claude v3 Sonnet`** to provide procedure codes for all relevant procedures listed in the plan generated by HealthScribe. This is something currently not supported by Comprehend Medical.

In [8]:
prompt = f"""

For each one of the bullet points defined in <plan></plan> XML tags between triple backslashes, 
list the most common CPT codes applicable. If there are no codes for a recommendation, do not include them in your response.

<plan>{plan}</plan>

"""

response_body = generate_message(prompt, 1000, 0)

print(response_body.get('content')[0].get('text'))

Here are the most common CPT codes applicable for each recommendation:

Lower back pain
- 97110 - Therapeutic procedure, therapeutic exercises
- 72100 - Radiologic examination, spine, lumbosacral; anteroposterior and lateral views

Sleep
- No applicable CPT codes

Exercise
- 97110 - Therapeutic procedure, therapeutic exercises

Follow up
- 72132 - Computed tomography, lumbar spine; with contrast material
- 72131 - Computed tomography, lumbar spine; without contrast material
